In [1]:
import os
import json
import pandas as pd
import traceback

In [30]:
!pip install langchain_community


In [32]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
dotenvobj = load_dotenv()
#os.getenv(OPENAI_API_KEY)
KEY = os.getenv("OPENAI_API_KEY")
#print (KEY)


In [37]:
llm = ChatOpenAI(openai_api_key = KEY, model_name = "gpt-3.5-turbo", temperature = 1)

In [64]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback

import PyPDF2

MCQ Generator : MCQ model is where GPT based on the given paragraph/pdf it will generate MCQs

In [39]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given a text, please create a quiz of {number} multiple choice questions
for {subject} students in {tone} tone.
Use json a response format.
"""

prompt = PromptTemplate (
    input_variables = ["text", "number", "subject", "tone"],
    template = TEMPLATE
)

In [85]:
mcq = LLMChain ( llm=llm, prompt=prompt, output_key="quiz", verbose=True)
result = mcq.run({
    "text": "Who won the Olympics gold medal in 400m running men?",
    "subject": "Olympics",
    "number": "4",
    "tone": "simple"
})




> Entering new LLMChain chain...
Prompt after formatting:

Text:Who won the Olympics gold medal in 400m running men?
You are an expert MCQ maker. Given a text, please create a quiz of 4 multiple choice questions
for Olympics students in simple tone.
Use json a response format.


> Finished chain.


In [71]:
print(result)

{
  "question": "Who won the Olympics gold medal in 400m running men?",
  "options": {
    "A": "Usain Bolt",
    "B": "Wayde van Niekerk",
    "C": "Michael Johnson",
    "D": "LaShawn Merritt"
  },
  "correct_answer": "B"
}


In [117]:
print(type(result))
print(result)



<class 'str'>
{
  "question1": {
    "question": "Who won the gold medal in men's 400m running at the Olympics?",
    "options": [
      "Usain Bolt",
      "Wayde van Niekerk",
      "Michael Johnson",
      "Carl Lewis"
    ],
    "answer": "Wayde van Niekerk"
  },
  "question2": {
    "question": "In which year did the athlete win the gold medal in the men's 400m running at the Olympics?",
    "options": [
      "2000",
      "2008",
      "2012",
      "2016"
    ],
    "answer": "2016"
  },
  "question3": {
    "question": "Which country does the gold medalist Wayde van Niekerk represent?",
    "options": [
      "Jamaica",
      "United States",
      "South Africa",
      "Canada"
    ],
    "answer": "South Africa"
  },
  "question4": {
    "question": "What is the distance of the race for which Wayde van Niekerk won the gold medal at the Olympics?",
    "options": [
      "100m",
      "200m",
      "400m",
      "800m"
    ],
    "answer": "400m"
  }
}


In [120]:
if isinstance(result, dict):
    print(result["quiz"])

In [111]:
TEMPLATE2 = """
    Given multiple choice quiz for {subject} students.
    Update the quiz questions as per the analytical abilities of the student
    change the tone that perfectly fits the students ability
    Quiz_MCQs = {quiz}

"""



In [112]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables = ["subject", "quiz"],
    template = TEMPLATE2
)

In [113]:
quiz_evaluation_prompt

PromptTemplate(input_variables=['quiz', 'subject'], template='\n    Given multiple choice quiz for {subject} students.\n    Update the quiz questions as per the analytical abilities of the student\n    change the tone that perfectly fits the students ability\n    Quiz_MCQs = {quiz}\n\n')

In [114]:
#Create Quiz evaluation chain,  llm=llm, prompt=prompt, output_key="quiz", verbose=True
review_chain = LLMChain (llm = llm, prompt = quiz_evaluation_prompt, output_key = "review", verbose = True)

In [122]:
review_chain.run({
    "subject": "olympics",
    "quiz": result
})



> Entering new LLMChain chain...
Prompt after formatting:

    Given multiple choice quiz for olympics students.
    Update the quiz questions as per the analytical abilities of the student
    change the tone that perfectly fits the students ability
    Quiz_MCQs = {
  "question1": {
    "question": "Who won the gold medal in men's 400m running at the Olympics?",
    "options": [
      "Usain Bolt",
      "Wayde van Niekerk",
      "Michael Johnson",
      "Carl Lewis"
    ],
    "answer": "Wayde van Niekerk"
  },
  "question2": {
    "question": "In which year did the athlete win the gold medal in the men's 400m running at the Olympics?",
    "options": [
      "2000",
      "2008",
      "2012",
      "2016"
    ],
    "answer": "2016"
  },
  "question3": {
    "question": "Which country does the gold medalist Wayde van Niekerk represent?",
    "options": [
      "Jamaica",
      "United States",
      "South Africa",
      "Canada"
    ],
    "answer": "South Africa"
  },
  "ques

'Quiz_MCQs = {\n  "question1": {\n    "question": "Who emerged victorious in the men\'s 400m race at the Olympics, showcasing unparalleled speed and determination?",\n    "options": [\n      "Usain Bolt",\n      "Wayde van Niekerk",\n      "Michael Johnson",\n      "Carl Lewis"\n    ],\n    "answer": "Wayde van Niekerk"\n  },\n  "question2": {\n    "question": "Can you recall the year when the talented athlete clinched the gold medal in the men\'s 400m event at the Olympics?",\n    "options": [\n      "2000",\n      "2008",\n      "2012",\n      "2016"\n    ],\n    "answer": "2016"\n  },\n  "question3": {\n    "question": "From which nation does the remarkable gold medalist Wayde van Niekerk proudly represent on the global stage?",\n    "options": [\n      "Jamaica",\n      "United States",\n      "South Africa",\n      "Canada"\n    ],\n    "answer": "South Africa"\n  },\n  "question4": {\n    "question": "Do you know the exact distance of the fierce race that propelled Wayde van Niek

In [123]:
# Now create a sequential chain

mcq_evaluate_chain = SequentialChain (
    chains = [mcq, review_chain],
    input_variables = ["text", "number", "subject", "tone" ],
    output_variables = ["quiz", "review"],
    verbose = True
    
)

How to track token usage tracking, using Get OpenAI callback,, with this we get Input Token Number, Output Token Number
Search for Token_usage_tracking on python.langchain.com

In [124]:
# Use callback if needed
with get_openai_callback() as cb:
    try:
        # Apply the chain
        response = mcq_evaluate_chain({
            "text": "Who won the Olympics gold medal in 400m running men?",
            "subject": "Olympics",
            "number": "4",
            "tone": "simple"
        })

    except ValueError as e:
        print("Error:", e)

print("Response:", response)
print("++++++++++++++++++++++++++++++++++++")

# Print callback info if needed
print("Callback info:", cb)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Who won the Olympics gold medal in 400m running men?
You are an expert MCQ maker. Given a text, please create a quiz of 4 multiple choice questions
for Olympics students in simple tone.
Use json a response format.


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    Given multiple choice quiz for Olympics students.
    Update the quiz questions as per the analytical abilities of the student
    change the tone that perfectly fits the students ability
    Quiz_MCQs = {
  "question": "Who won the Olympics gold medal in 400m running men?",
  "options": [
    {"A": "Usain Bolt"},
    {"B": "Wayde van Niekerk"},
    {"C": "Michael Johnson"},
    {"D": "Carl Lewis"}
  ],
  "answer": "B"
}



> Finished chain.

> Finished chain.
Response: {'text': 'Who won the Olympics gold medal in 400m running men?', 'subject': 'Olympics', 'number': '4', 'tone': 'simple

In [125]:
print("Response:", response)
print("++++++++++++++++++++++++++++++++++++")

print(response["quiz"])

print("++++++++++++++++++++++++++++++++++++")

print(response["review"])

# Print callback info if needed
print("Callback info:", cb)

Response: {'text': 'Who won the Olympics gold medal in 400m running men?', 'subject': 'Olympics', 'number': '4', 'tone': 'simple', 'quiz': '{\n  "question": "Who won the Olympics gold medal in 400m running men?",\n  "options": [\n    {"A": "Usain Bolt"},\n    {"B": "Wayde van Niekerk"},\n    {"C": "Michael Johnson"},\n    {"D": "Carl Lewis"}\n  ],\n  "answer": "B"\n}', 'review': 'Quiz_MCQs = {\n  "question": "In the 2016 Olympics, who set a new world record in the 400m running men event?",\n  "options": [\n    {"A": "Usain Bolt"},\n    {"B": "Wayde van Niekerk"},\n    {"C": "Michael Johnson"},\n    {"D": "Carl Lewis"}\n  ],\n  "answer": "B"\n}'}
++++++++++++++++++++++++++++++++++++
{
  "question": "Who won the Olympics gold medal in 400m running men?",
  "options": [
    {"A": "Usain Bolt"},
    {"B": "Wayde van Niekerk"},
    {"C": "Michael Johnson"},
    {"D": "Carl Lewis"}
  ],
  "answer": "B"
}
++++++++++++++++++++++++++++++++++++
Quiz_MCQs = {
  "question": "In the 2016 Olympics, 

In [132]:
print("++++++++++++++++++++++++++++++++++++")

print(response["quiz"])

++++++++++++++++++++++++++++++++++++
{
  "question": "Who won the Olympics gold medal in 400m running men?",
  "options": [
    {"A": "Usain Bolt"},
    {"B": "Wayde van Niekerk"},
    {"C": "Michael Johnson"},
    {"D": "Carl Lewis"}
  ],
  "answer": "B"
}
